In [1]:
import numpy as np
import pandas as pd
import pickle

test_preprocessed = pd.read_csv('test_preprocessed.csv', index_col = 0)

test_preprocessed
X_test = test_preprocessed.drop('id',axis=1)
X_test = X_test.values

In [5]:
class StackingClassifer():
    def __init__(self, estimators, estimators_second, merge_estimator, ):
        
        self.original_clfs = dict(estimators)
        self.original_clfs_second = dict(estimators_second)

        self.m_clf = merge_estimator
        
        self.clf_dict = defaultdict(list)
        self.clf_dict_second = defaultdict(list)
        self.clfs_index = sorted(self.original_clfs.keys())
        self.clfs_index_second = sorted(self.original_clfs_second.keys())

        
    def fit(self, X,y):
        #リストの初期化
        self.clfs_dict =  defaultdict(list) 
        #交差検証(k=5)
        K = 5
        kf = KFold(n_splits=K, shuffle=True, random_state=1 )

        #indexのリストを作成
        index_list = list(kf.split(X, y))
        
        #第一段の学習
        #第一段の予測を次の特徴量としてリストにする
        merge_feature_list = []
        # 初期の学習器で学習を行う
        count = 0
        for clf_name in self.clfs_index:
            #学習器をセット
            clf_origin = self.original_clfs[clf_name]
            #予測のりすと
            preds_temp_list = []
            #交差検証実施
            for train_index, test_index in index_list:
                #この時点で学習器をコピー
                clf_copy = deepcopy(clf_origin)
                #学習スタート
                clf_copy.fit(X[train_index], y[train_index])
                #予測のリストを格納
                preds_temp_list.extend(
                (clf_copy.predict(X[test_index])).tolist())
#                 print(preds_temp_list)
            
                #学習ずみのモデルを格納
                self.clf_dict[clf_name].append(clf_copy)
            if count == 0:
                merge_feature_list = preds_temp_list
                merge_feature_list  = np.array(merge_feature_list).reshape(-1,1)
                
            else:
                preds_temp_list = np.array(preds_temp_list).reshape(-1,1)
                merge_feature_list = np.concatenate((merge_feature_list, preds_temp_list), axis=1)

        #予測のリストを次の学習の特徴量とする.
#             preds_temp_list = np.array(preds_temp_list)
#             merge_feature_list.append(preds_temp_list)
            count += 1

#         print(len(merge_feature_list))
        merge_feature_list = np.array(merge_feature_list)
        print('第一段階の特徴量のサイズ:{0}'.format(merge_feature_list.shape))
        
#         #Xも特徴量に追加 <--これは削除しても良い？
#         merge_feature_list = np.concatenate((merge_feature_list, X), axis=1)
#         print(merge_feature_list.shape)
#         print()
        
    
        #第二学習フェーズ
        y_merged_second = np.hstack([y[test_index]
                              for _, test_index in index_list])
        
        #リストの初期化
        self.clfs_dict_second =  defaultdict(list) 
        #交差検証(k=5)
        K = 5
        kf = KFold(n_splits=K, shuffle=True, random_state=1 )
        #indexのリストを作成
        index_list = list(kf.split(merge_feature_list, y_merged_second))
        
        merge_feature_list_second = []
        # 初期の学習器で学習を行う
        count = 0
        for clf_name in self.clfs_index_second:
            #学習器をセット
            clf_origin_second = self.original_clfs_second[clf_name]
            #予測のりすと
            preds_temp_list_second = []
            #交差検証実施
            for train_index, test_index in index_list:
                #この時点で学習器をコピー
                clf_copy_second = deepcopy(clf_origin_second)
                #学習スタート
                clf_copy_second.fit(merge_feature_list[train_index], y_merged_second[train_index])
                #予測のリストを格納
                preds_temp_list_second.extend(
                (clf_copy_second.predict(merge_feature_list[test_index])).tolist())
#                 print(preds_temp_list)
            
                #学習ずみのモデルを格納
                self.clf_dict_second[clf_name].append(clf_copy_second)
            if count == 0:
                merge_feature_list_second = preds_temp_list_second
                merge_feature_list_second  = np.array(merge_feature_list_second).reshape(-1,1)
                
            else:
                preds_temp_list_second = np.array(preds_temp_list_second).reshape(-1,1)
                merge_feature_list_second = np.concatenate((merge_feature_list_second, preds_temp_list_second), axis=1)

        #予測のリストを次の学習の特徴量とする.
#             preds_temp_list = np.array(preds_temp_list)
#             merge_feature_list.append(preds_temp_list)
            count += 1

#         print(len(merge_feature_list))
            merge_feature_list_second = np.array(merge_feature_list_second)
#         print(merge_feature_list)

        print('第二段階の特徴量のサイズ:{0}'.format(merge_feature_list_second.shape))
    
        #第三学習フェーズ
        X_merged = merge_feature_list_second
        
        print(X_merged.shape)
        
#         print(X_merged.shape)
        y_merged = np.hstack([y_merged_second[test_index]
                              for _, test_index in index_list])
        print(y_merged.shape)
        
        self.m_clf.fit(X_merged, y_merged)
        
        return self
    
    def predict(self, X):
        #第一段階
        #特徴量のリストを作成
        merge_feature_for_test = []
        # それぞれの学習器で学習
        count = 0
        for clf_name in self.clfs_index:
        #予測のリストを作成
            print(clf_name)
            temp_proba_list = []
            # 各学習済みモデルを引き出す
            for clf in self.clf_dict[clf_name]:
                #予測のリストに各学習器の予測を追加
                temp_proba_list.append(clf.predict(X))
            # 予測を行方向に平均をとる。
            merge_feature_for_test.append(
                np.mean(temp_proba_list, axis=0))
            
        print('(予測)第一段階の特徴量のサイズ:{0}'.format(np.array(merge_feature_for_test).T))
        

            
        #第二段階
        #特徴量のリストを作成
        X_merged_second = np.array(merge_feature_for_test).T
#         #Xも特徴量に追加 <--これは削除しても良い？
#         X_merged_second = np.concatenate((X_merged_second, X), axis=1)
        print(X_merged_second.shape)
        merge_feature_for_test_second = []
        # それぞれの学習器で学習
        count = 0
        for clf_name in self.clfs_index_second:
            print(clf_name)
        #予測のリストを作成
            temp_proba_list_second = []
            # 各学習済みモデルを引き出す
            for clf in self.clf_dict_second[clf_name]:
                #予測のリストに各学習器の予測を追加
                temp_proba_list_second.append(clf.predict(X_merged_second))
            # 予測を行方向に平均をとる。
            merge_feature_for_test_second.append(
                np.mean(temp_proba_list_second, axis=0))
            
        print('(予測)第二段階の特徴量のサイズ:{0}'.format(np.array(merge_feature_for_test_second).T))

            
        # 特徴量リストを
        X_merged = np.array(merge_feature_for_test_second).T
        print(X_merged.shape)
        print(X_merged[:,0])
        X_merged_harmonic_mean = 1/((1/X_merged[:,0]  + 1/X_merged[:, 1]  + 1/X_merged[:,2] ) * (1/3))
        print(X_merged_harmonic_mean)
        print(X_merged_harmonic_mean.shape)
        predict_X = np.average(X_merged, axis=1)
        print(predict_X)      
        print(np.exp(predict_X))



        return X_merged_harmonic_mean
            

In [7]:
with open('model_stcl_develop.pickle', mode='rb') as fp:
    clf_stcl = pickle.load(fp)
print(test_preprocessed['id'].shape)
test_preprocessed['y'] =pd.DataFrame(np.exp(clf_stcl.predict(X_test)))
# print(test_preprocessed['y'].shape)
df_pred_stcl = pd.concat([ test_preprocessed['id'],test_preprocessed['y']], axis=1)
df_pred_stcl[['id','y']].to_csv('pred_by_stcl_develop.csv', index = None, header=None)
df_pred_stcl

/Users/wata_ruja/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/wata_ruja/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``bre

(477,)
Bridge
est
lgb
rf
ridge
xgb
(予測)第一段階の特徴量のサイズ:[[10.43853948 10.488854   10.49221513 10.49243449 10.37736251 10.25141716]
 [ 9.51058694  9.76244001  9.75980945  9.67485966  9.5395018   9.53824902]
 [ 9.7599778   9.95632372 10.01368075  9.80174096  9.75521081  9.79239464]
 ...
 [ 9.39567556  9.70861524  9.63715697  9.59555861  9.44152599  9.56958485]
 [ 9.57209566  9.80270428  9.86913063  9.83327606  9.58029691  9.64188576]
 [ 9.57496033  9.58015456  9.57882671  9.4919504   9.579279    9.47570324]]
(477, 6)
lgb
ridge
xgb
(予測)第二段階の特徴量のサイズ:[[10.56214095 10.32486629 10.50498867]
 [ 9.77427287  9.64718267  9.6995182 ]
 [10.05820724  9.82940096  9.9385767 ]
 ...
 [ 9.68337835  9.57981754  9.61515903]
 [ 9.93781192  9.72002338  9.83293438]
 [ 9.6614769   9.57070442  9.58448982]]
(477, 3)
[10.56214095  9.77427287 10.05820724  9.70063663  9.8326661   9.59737189
 10.19165042  9.34328435 10.14622716 10.7137019   9.53465878  9.70977304
 10.14139501  9.296636    9.67290502  9.49309363  9.60201

,id,y
0,19075,34996.960098
1,19076,16427.488869
2,19077,20768.315714
3,19078,15412.513027
4,19079,17453.639430
5,19080,14048.971709
6,19081,24320.552510
7,19082,11627.932372
8,19083,23112.020460
9,19084,38223.342113
